In [1]:
from pyspark.sql import SparkSession

import getpass
username = getpass.getuser()

spark = SparkSession. \
    builder. \
    config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1'). \
    config('spark.ui.port', '0'). \
    config('spark.sql.warehouse.dir', f'/user/{username}/warehouse'). \
    enableHiveSupport(). \
    appName(f'{username} | Python - Kafka and Spark Integration'). \
    master('yarn'). \
    getOrCreate()

In [2]:
kafka_bootstrap_servers = 'w01.itversity.com:9092,w02.itversity.com:9092'

In [3]:
df = spark. \
  readStream. \
  format('kafka'). \
  option('kafka.bootstrap.servers', kafka_bootstrap_servers). \
  option('subscribe', f'{username}_retail'). \
  load()

In [4]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



We can use below code snippet using CLI to preview the data using `console`.
```python
df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)"). \
    writeStream. \
    outputMode("update"). \
    format("console"). \
    option('truncate', 'false'). \
    trigger(processingTime='5 seconds'). \
    start()
```

In [5]:
from pyspark.sql.functions import date_format, to_date, split, lit, substring, length

In [ ]:
df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)"). \
    withColumn('year', to_date(substring(split('value', ' ')[3], 2, 21), '[dd/MMM/yyyy:HH:mm:ss')). \
    writeStream. \
    format("memory"). \
    queryName("log_messages"). \
    start()

In [16]:
spark.sql('SELECT * FROM log_messages').show(truncate=False)

+----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+
|key |value                                                                                                                                                                                                                                         |year      |
+----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+
|null|182.91.126.193 - - [23/Feb/2023:22:18:31 -0800] "GET /department/golf/categories HTTP/1.1" 404 969 "-" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.77.4 (KHTML, like Gecko) Version/7.0.5 Safari/537.77.4" 

In [18]:
spark.sql('SELECT count(1) FROM log_messages').show(truncate=False)

+--------+
|count(1)|
+--------+
|2000    |
+--------+

